In [20]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from PIL import Image
import copy
import numpy as np

In [15]:
#load meta information
files = pd.read_csv("Dataset_FollowUp/metadata.csv")
imgpath = "Dataset_FollowUp/"+files['folder']+'/'+files['filename']
#number of CXR images
N = 259

In [16]:
device = torch.device("cuda:0")

In [17]:
num_classes = 3
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier[1] = nn.Linear(1280,num_classes)
mobilenet = torch.load('MobileNet_weights/best_validation_weights')
mobilenet.to(device)
mobilenet.eval();
feat = copy.deepcopy(mobilenet)
feat.classifier = feat.classifier[0]
result_path = []
result_feats = []
feat.eval();
for i in range(N):
    img_path = imgpath[i]
    img = Image.open(img_path)
    img = img.convert("RGB")
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    transform =    transforms.Compose([
            transforms.Resize((256,256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize])

    img = transform(img)
    with torch.no_grad():
        img = img.unsqueeze(0)
        img = img.cuda()
        outputs = feat(img)
        output_np = outputs.cpu().numpy().squeeze()
        result_path.append(img_path)
        result_feats.append(output_np)

In [ ]:
result_feats_np = np.array(result_feats)
np.save('MobileNet_Features.npy',result_feats_np)